In [7]:
# Raster calculator for notebook. If you have images that have stack image(3 bands in 1 image); 
#first, read image band by band and give the different band for input 
#ds1 = gdal.Open(nir, GA_ReadOnly )
#band1 = ds1.GetRasterBand(1) #red band 
#band2=ds1.GetRasterBand(4)   #nir band 

In [237]:
from osgeo import gdal
from osgeo.gdalnumeric import *
from osgeo.gdalconst import *
import pandas as pd
import numpy as np
import time

In [1]:
def ListofExtensionAndName(directory,extension):
     
        if len(directory) != None:
            import os
            FilesList = []
            FileName=[]
            for root, subdirectory, files in os.walk(directory):
                for file in files:
                    if file.endswith(extension):
                        FilesList.append(os.path.join(root,file))
                        base=os.path.basename(file)
                        FileName.append(os.path.splitext(base)[0])

            return FilesList,FileName
        else:
            print("no"+ extension +"file for this directory")

In [233]:
nir_list=ListofExtensionAndName('','TIF')


In [234]:
red_list=ListofExtensionAndName('','TIF')


In [ ]:
start_time = time.time()
for index,(red,nir) in enumerate(zip(red_list[0],nir_list[0]),0):

    #Open the dataset
    #NIR band
    ds1 = gdal.Open(nir, GA_ReadOnly )
    band1 = ds1.GetRasterBand(1)
    #red band
    ds2 = gdal.Open(red, GA_ReadOnly )
    band2=ds2.GetRasterBand(1)
    
    #Read the data into numpy arrays
    data1 = BandReadAsArray(band1)
    data1=data1.astype(float32)
    data2 = BandReadAsArray(band2)
    data2=data2.astype(float32)
    
    #Allow 0 division in numpy
    np.seterr(divide='ignore', invalid='ignore')
    check = np.logical_or ( data1 > 0, data2 > 0 )
    dataOut = np.empty(data1.shape, dtype=np.float32)
    #The actual calculation
    #NDVI
    dataOut = np.where ( check,  (data1 - data2 ) / ( data1 + data2 ), 0 )
    
    output_path=r'../data/sample.tif'
    
    #define nodata value for output image
    NoData_value = 0
    #Write the out file
    driver = gdal.GetDriverByName("GTiff")
    dsOut = driver.Create(output_path, ds1.RasterXSize, ds1.RasterYSize, 1, gdal.GDT_Float32)
    CopyDatasetInfo(ds1,dsOut)
    bandOut=dsOut.GetRasterBand(1)
    bandOut.SetNoDataValue(NoData_value)
    BandWriteArray(bandOut, dataOut)
    #Close the datasets
    band1 = None
    band2 = None
    ds1 = None
    bandOut = None
    dsOut = None
    
    
print("--- %s seconds ---" % (time.time() - start_time))
